## 什么是rag
检索增强生成（Retrieval-augmented Generation），简称RAG，是当下热门的大模型前沿技术之一。

检索增强生成模型结合了语言模型和信息检索技术。具体来说，当模型需要生成文本或者回答问题时，它会先从一个庞大的文档集合中检索出相关的信息，然后利用这些检索到的信息来指导文本的生成，从而提高预测的质量和准确性。

2020年，Facebook AI Research(FAIR)团队发表名为《Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks》的论文。该篇论文首次提出了RAG概念。

其核心流程就是根据用户提问，从私有知识中检索到“包含答案的内容”，然后把“包含答案的内容”和用户提问一起放到prompt（提示词）中，提交给大模型，此时大模型的回答就会充分考虑到“包含答案的内容”，看起来也就更“懂”我们。

## RAG的总体工作流程

<img src="rag.png" style="margin-left: 0px" width="800px">

## RAG重点技术拆解
想要根据用户的提问去找我们的海量知识库找相关内容，是RAG流程中最重要的一环。

如何比较语义的相关性，实现相关知识的召回。

“脑残”和“二货”有语义关系性，虽然他们表示的可能是同一类人。但是传统的数据库查询也好，搜索引擎也好，没办法理解他们的关系，我们都是基于关键字搜索来查询相关的数据。所以传统的应用需要人为的将“脑残”和“二货”等词之间打上特征标签进行关联，这样才能实现语义搜索。生成和挑选特征这个过程，也被称为 Feature Engineering (特征工程)，如果词汇量少可以通过打标签实现相同语义，但是当文本数据量太多的时候，特征就会变得很多了，尤其是还有图片数据、语音视频等。而且很难人为的进行标注，这时候需要一种自动化的方式来提取这些特征，而这可以通过 Vector Embedding 实现。

### Embedding登场

文本向量化（词嵌入），并比较两个向量在多为空间是否相近，相近我们认为他们有语义相关性。一般有两种方式计算：
<img src="sim.png" style="margin-left: 0px" width="400px">

(1) 欧几里得距离(Euclidean distance)：测量点之间的直接距离。适用于聚类或对整体差异很重要的密集特征集进行分类。

In [1]:
import numpy as np
from numpy import dot
from numpy.linalg import norm


def ouShiDistanceT(a, b):
    """
    计算两个向量的欧氏距离,越小越相似
    """
    return norm(np.asarray(a) - np.asarray(b))

(2) 余弦相似性(Cosine similarity)：关注向量之间的角度。非常适合文本处理和信息检索，根据方向而不是传统距离捕获语义相似性。

In [2]:
def cosine_similarity(a, b):
    """
    计算两个向量的余弦相似度,越大越相似
    """
    return dot(a, b)/(norm(a)*norm(b))

#### 两种相似度计算结果比较

In [3]:
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
import os

client = OpenAI()

def _get_embedding(text,model="text-embedding-ada-002"):
    """
    获取文本的嵌入向量
    """
    response = client.embeddings.create(
        input=text,
        model=model
    ).data
    return [x.embedding for x in response]

In [4]:
query = "我在学习python，我发现它和java很像，都是把一堆东西放在一起，然后希望它不会爆炸。"

documents = [
    "最近在搞大语音模型开发，自信心受到了亿点点伤害",
    "人生就像一盒巧克力，你永远不知道下一颗是什么味道，尤其是当它过期的时候",
    "好基友桃园三结义，林冲雪夜上梁山，林黛玉倒拔垂杨柳",
    "我愿意为你写一辈子的代码，直到我的内存溢出"
]
query_vec = _get_embedding(query)[0]
doc_vecs = _get_embedding(documents)

print("余弦距离，越大越相似:")  # 越大越相似
print(cosine_similarity(query_vec, query_vec))
for vec in doc_vecs:
    print(cosine_similarity(query_vec, vec))



print("欧式距离，越小越相似:")  # 越小越相似
print(ouShiDistanceT(query_vec, query_vec))
for vec in doc_vecs:
    print(ouShiDistanceT(query_vec, vec))

余弦距离，越大越相似:
1.0
0.7921531592606031
0.7701232256205532
0.7443536173757629
0.8233306181894029
欧式距离，越小越相似:
0.0
0.6447431091040607
0.6780513198515167
0.7150474331871133
0.5944230627828705


那么如何把文本变成向量？

## 嵌入模型

什么是嵌入模型(Embedding Model)？
嵌入模型将各种数据（例如文本、图像、图表和视频）转换为数字向量，从而在多维向量空间中捕捉其含义和细微差别

<img src="embeddings.png" style="margin-left: 0px" width=800px>

https://huggingface.co/spaces/mteb/leaderboard

<img src="embedding-models.png" style="margin-left: 0px" width=1000px>

指标：
- classification：分类，使用准确率或精确率指标。
- clustering： 聚类，指标使用V-Measure
- pair classification：给定一对文本，判断其是否具有相同含义，使用AP指标
- reranking：重排集合与query 的相关性，使用MAP指标
- retrieval：相关性召回，在无限集合中查询，使用NDCG@10指标
- STS： 相似性，大部分使用cosine 相似

## 向量数据库

对于传统数据库，基于文本的精确匹配，这种索引和搜索算法对于关键字的搜索功能非常合适，但对于语义搜索功能就非常弱。基于向量的相似性搜索(Similarity Search)实现的数据库查询就是我们称为向量数据库。

**向量数据库中的相似性搜索**

相似性搜索，也称为向量搜索、向量相似性或语义搜索，根据指定的相似性度量从数据库中有效检索与给定查询的向量嵌入在语义上相似的向量的过程


In [5]:
import chromadb
from chromadb.config import Settings

class VectorDBConnector:
    def __init__(self, collection_name, embeding_fn):
        chroma_client = chromadb.HttpClient(host="10.104.60.47", port=8000, settings=Settings(allow_reset=True))

        # 创建一个 collection
        self.collection = chroma_client.get_or_create_collection(name=collection_name)
        self.embeding_fn = embeding_fn

    def add_data(self, documents):
        # 添加数据到集合中，包括文档的嵌入向量、原始文档和文档id
        print("开始添加数据")
        self.collection.add(
            embeddings=self.embeding_fn(documents),  # 文档嵌入向量
            documents=documents,
            ids=[f"id{i}" for i in range(len(documents))]  # 文档id
        )

    def search_data(self, query, top_n):
        """在集合中搜索数据。"""
        results = self.collection.query(
            query_embeddings=self.embeding_fn(query),
            n_results=top_n)
        print("========向量数据库搜索结果========" )
        print(results)
        return results

In [6]:
import re
def split_text_into_paragraphs(file_path):
    """
    按段落分割文本文件。

    :return: 包含所有段落的列表
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            # 读取整个文件内容
            content = file.read()

            # 使用正则表达式分割段落，\n\n代表两个换行符之间的空行
            paragraphs = re.split(r'\n\n+', content)

            # 移除空字符串（如果有的话）
            paragraphs = [p for p in paragraphs if p.strip()]

            return paragraphs
    except FileNotFoundError:
        print(f"文件 {self.file_path} 未找到")
    except IOError as e:
        print(f"读取文件时发生错误: {e}")
    except Exception as e:
        print(f"未知错误: {e}")

In [7]:
vectordb = VectorDBConnector("test", embeding_fn=_get_embedding)
file_path = "./体检知识库.txt"
documents = split_text_into_paragraphs(file_path)

def test_add_doc_todb():
    print("添加文档到数据库")
    vectordb.add_data(documents)
def testvectodb():
    print('检索文档数据库')
    result = vectordb.search_data(query, top_n=1)
    for doc in result['documents'][0]:
        print(doc + "\n")

## 大模型结合向量搜索实现让大模型更加聪明

In [8]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())  


client = OpenAI()

prompt_template = """
你时一个问答机器人，工作在温附一的体检中心，负责回答体检患者问的问题。你的任务时根据下述给定的已知信息回答用户问题，
确保你的回答完全符合依据下述已知的信息，不要编造答案，语气一定要客气。如果下述已知信息不足以回答用户问题，
请直接答复：“抱歉，你触及到了我的知识盲区，暂时无法回答你的问题”
已知信息:
__INFO__
用户问：
__QUERY__
请用中文回答用户问题
"""

def _get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,  # 模型输出的随机性，0 表示随机性最小
    )
    return response.choices[0].message.content

def _get_embedding(text, model="text-embedding-ada-002"):
    """
    获取文本的嵌入向量
    """
    response = client.embeddings.create(input=text ,model=model).data
    return [x.embedding for x in response]

def build_prompt(**kwargs):
    prompt = prompt_template
    for key, value in kwargs.items():
        if isinstance(value, str):
            val = value
        elif isinstance(value, list) and all(isinstance(elem, str) for elem in value):
            val = "\n".join(value)
        else:
            val = str(value)
        prompt = prompt.replace(f"__{key.upper()}__", val)
    return prompt

vectordb = VectorDBConnector("test", embeding_fn=_get_embedding)

def chat(userQuery, top_n = 2):
    # 根据用户检索数据
    search_results = vectordb.search_data(userQuery, top_n)
    prompt = build_prompt(INFO=search_results['documents'][0], QUERY=userQuery)
    response = _get_completion(prompt)
    return response

In [9]:
response = chat(userQuery="空腹可以体检吗")
print("========rag response========")
print( response)

========向量数据库搜索结果========
{'ids': [['id16', 'id12']], 'distances': [[0.29690515995025635, 0.30947327613830566]], 'embeddings': None, 'metadatas': [[None, None]], 'documents': [['为什么空腹的项目不能先做？\n抱歉，目前空腹的项目人数较多，需要排队较长时间，我们由智能导检分配到排队较少的项目，可以加快您体检的进程', '这个体检套餐全面了吗？\n您好，这个体检套餐的内容是根据您的整体情况定制的，根据这个体检套餐的内容先进行初步筛查，如果有后续的问题，可以进一步深入检查。']], 'uris': None, 'data': None, 'included': ['metadatas', 'documents', 'distances']}
========rag response========
抱歉，空腹是为了某些项目需要，如果您的体检项目不需要空腹，是可以进行体检的。如果您有任何疑问或需要进一步了解，请随时告诉我们。祝您体检顺利！


## 开源RAG介绍和演示
https://github.com/langgenius/dify

https://github.com/QuivrHQ/quivr

https://github.com/chatchat-space/Langchain-Chatchat

https://github.com/Mintplex-Labs/anything-llm

https://github.com/netease-youdao/QAnything